# Calculating DRIS index

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

## Data Processing

### Dummy Data

In [44]:
data_diagnosed_dict = {
    'P': [12,11,10, 9],
    'Mg': [2,2,3,3],
    'N': [1,2,1,2],
    'Ca': [3,4,3,3],
    'Mn': [4,5,4,2]
}

data_optimum_dict = {
    'p': [12,11,10],
    'mg': [2,1,3],
    'n': [2,1,1],
    'ca': [4,4,3],
    'mn': [5,4,4],
}

df_diagnosed = pd.DataFrame(data_diagnosed_dict)
df_optimum = pd.DataFrame(data_optimum_dict)

df_diagnosed.head()

,P,Mg,N,Ca,Mn
0,12,2,1,3,4
1,11,2,2,4,5
2,10,3,1,3,4
3,9,3,2,3,2


### Calculate ratios

In [45]:
def calculate_ratios(df):
    # Step 2: Calculate the ratios
    ratios = {}
    for col1, col2 in combinations(df.columns, 2):
        ratio_name1 = f"{col1}/{col2}"
        # ratio_name2 = f"{col2}/{col1}"
        ratios[ratio_name1] = df[col1] / df[col2]
        # ratios[ratio_name2] = df[col2] / df[col1]
    return pd.DataFrame(ratios)


df_optimum_ratios = calculate_ratios(df_optimum)
df_diagnosed_ratios = calculate_ratios(df_diagnosed)

df_diagnosed_ratios.head()

,P/Mg,P/N,P/Ca,P/Mn,Mg/N,Mg/Ca,Mg/Mn,N/Ca,N/Mn,Ca/Mn
0,6.000000,12.0,4.000000,3.0,2.0,0.666667,0.50,0.333333,0.25,0.75
1,5.500000,5.5,2.750000,2.2,1.0,0.500000,0.40,0.500000,0.40,0.80
2,3.333333,10.0,3.333333,2.5,3.0,1.000000,0.75,0.333333,0.25,0.75
3,3.000000,4.5,3.000000,4.5,1.5,1.000000,1.50,0.666667,1.00,1.50


### Calculate coefficient of variation of the optimum values

In [54]:
def calculate_CV(df_ratios):
    cv_dict = {}
    for ratio_name, ratio_values in df_ratios.items():
        mean_ratio = ratio_values.mean()
        std_ratio = ratio_values.std()
        cv = (std_ratio / mean_ratio) * 100  # percentage or not 
        cv_dict[ratio_name] = [cv]
    return pd.DataFrame(cv_dict)

df_optimum_CV = calculate_CV(df_optimum_ratios)

,p/mg,p/n,p/ca,p/mn,mg/n,mg/ca,mg/mn,n/ca,n/mn,ca/mn
0,57.423868,29.397237,9.665737,7.069708,69.282032,65.465367,54.981444,35.250582,28.867513,15.563243


### Calculate rations of the mean

In [62]:
df_op = pd.DataFrame(df_optimum.mean()).T
df_optimum_mean_ratios = calculate_ratios(df_op)

df_di = pd.DataFrame(df_diagnosed.mean()).T
df_diagnosed_mean_ratios = calculate_ratios(df_di)

df_diagnosed_mean_ratios.head()

,P/Mg,P/N,P/Ca,P/Mn,Mg/N,Mg/Ca,Mg/Mn,N/Ca,N/Mn,Ca/Mn
0,4.2,7.0,3.230769,2.8,1.666667,0.769231,0.666667,0.461538,0.4,0.866667


In [63]:
df_optimum_mean_ratios.head()

,p/mg,p/n,p/ca,p/mn,mg/n,mg/ca,mg/mn,n/ca,n/mn,ca/mn
0,5.5,8.25,3.0,2.538462,1.5,0.545455,0.461538,0.363636,0.307692,0.846154


## Calculate f(A/B) 

\begin{align}
    f(A/B) = 
    \begin{cases}
    &\biggl(\frac{A/B}{a/b} - 1\biggr)\frac{1000}{CV} \quad A/B \ge a/b\\
    &\biggl(1 - \frac{a/b}{A/B}\biggr)\frac{1000}{CV} \quad A/B < a/b
    \end{cases}
\end{align}

- $A/B$ is the ratio of two elements (A and B) in the tissue of the pant being diagnosed.
- $a/b$ is the ration of the optimal values 
- CV is the coefficient of variation of the optimal values $a$ and $b$: $CV = \sigma/\mu$ 

In [48]:
def f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV):
    names = df_diagnosed_mean_ratios.columns
    diagnosed_ratios = df_diagnosed_mean_ratios.to_numpy()
    optimum_mean_ratios = df_optimum_mean_ratios.to_numpy()
    optimum_CV = df_optimum_CV.to_numpy()

    def f_single(diagnosed_ratio, optimum_mean_ratio, CV):
        if diagnosed_ratio == 0:
            return float('inf')  # Return inf or some large number to handle zero division
        if diagnosed_ratio >= optimum_mean_ratio:
            f = ((diagnosed_ratio / optimum_mean_ratio) - 1) * 1000 / CV
        else:
            f = (1 - (optimum_mean_ratio / diagnosed_ratio)) * 1000 / CV
        return [f]

    f_dict = dict()
    for i in range(len(names)):
        f_dict[f"f({names[i]})"] = f_single(diagnosed_ratios[0, i], optimum_mean_ratios[0, i], optimum_CV[0, i])
    return pd.DataFrame(f_dict)


# Example usage
# Assuming df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV are defined DataFrames with similar structure
df_f = f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV)
df_f.head()


,f(P/Mg),f(P/N),f(P/Ca),f(P/Mn),f(Mg/N),f(Mg/Ca),f(Mg/Mn),f(N/Ca),f(N/Mn),f(Ca/Mn)
0,-5.390159,-6.074429,7.958325,14.573487,1.603751,6.26677,8.083535,7.637626,10.392305,1.557672


## Calculate DRIS index


$\mathrm{A_{index}} = \frac{1}{z} \bigl[f(A/B) + f(A/C) + f(A/D) + \cdots + f(A/N)\bigr]$

In [67]:
def create_index_string(index_element, df_diagnosed):
    ''' Create string representing the the equation to calculate the DRIS index '''
    elements = df_diagnosed.columns
    result_string = f'I_{elements[index_element]} ='
    for i, element in enumerate(elements):
        if index_element < i:
            result_string += (f' + f({elements[index_element]}/{element})')
        elif index_element > i:
            result_string += (f' - f({element}/{elements[index_element]})')
    return result_string

result_string = create_index_string(2,df_diagnosed)
result_string

'I_N = - f(P/N) - f(Mg/N) + f(N/Ca) + f(N/Mn)'

In [68]:
def calculate_index_value(index_element, df_diagnosed, df_f):
    ''' Calculates the DRIS index '''
    f_dict = df_f.to_dict('index')[0]
    elements = df_diagnosed.columns
    result = 0
    for i, element in enumerate(elements):
        if index_element < i:
            result += f_dict[f'f({elements[index_element]}/{element})']
        elif index_element > i:
            result -= f_dict[f'f({element}/{elements[index_element]})']
    return result

In [69]:
def calculate_all_index_values(df_diagnosed, df_f):
    elements = df_diagnosed.columns
    results_dict = dict()
    for i in range(len(elements)):
        results_dict[f'I_{elements[i]}'] = [calculate_index_value(i, df_diagnosed, df_f)]
    return pd.DataFrame(results_dict)

DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)

DRIS_indices.head()

,I_P,I_Mg,I_N,I_Ca,I_Mn
0,11.067223,21.344216,22.500609,-20.305049,-34.606999


## Putting it together

In [53]:
def calculate_DRIS_index(df_diagnosed, df_optimum):
    # calculate ratios
    df_optimum_ratios = calculate_ratios(df_optimum)
    # df_diagnosed_ratios = calculate_ratios(df_diagnosed)
    # calculate CV
    df_optimum_CV = calculate_CV(df_optimum_ratios)
    # calculate mean _ratios
    df_op = pd.DataFrame(df_optimum.mean()).T
    df_optimum_mean_ratios = calculate_ratios(df_op)
    df_di = pd.DataFrame(df_diagnosed.mean()).T
    df_diagnosed_mean_ratios = calculate_ratios(df_di)
    # calculate f
    df_f = f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV)
    # calculate indices
    DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)
    return DRIS_indices

DRIS = calculate_DRIS_index(df_diagnosed, df_optimum)
DRIS.head()


,I_P,I_Mg,I_N,I_Ca,I_Mn
0,11.067223,21.344216,22.500609,-20.305049,-34.606999


## Nurtional Balance Index average (NBIa)

In [101]:
def calculate_NBIa(df_dris):
    """ This value is for the whole population """
    n = len(df_dris.values[0])
    return np.sum(np.abs(df_dris.values[0]))/n

def calculate_NBI(df_dris):
    """ This value is for the whole population """
    return np.sum(np.abs(df_dris.values))

calculate_NBIa(DRIS)

np.float64(21.964819284601006)

## Interpretation

In [115]:
def interpret(df_dris):
    NBIa = calculate_NBIa(df_dris)
    print(f"NBIa ={np.round(NBIa,2)}")
    for i, dris_index in enumerate(df_dris.values[0]):
        if dris_index < NBIa and dris_index < 0 and dris_index and dris_index == min(df_dris.values[0]):
            print(f"{df_dris.columns[i]} Deficiency")
        elif dris_index < NBIa and dris_index < 0:
            print(f"{df_dris.columns[i]} Deficiency-prone")
        elif np.round(dris_index,0) == np.round(NBIa,0):
            print(f"{df_dris.columns[i]} Sufficient")
        elif dris_index > NBIa and dris_index > 0 and dris_index == max(df_dris.values[0]):
            print(f"{df_dris.columns[i]} Excess")
        elif dris_index > NBIa and dris_index > 0:
            print(f"{df_dris.columns[i]} Excess-prone")
        elif dris_index > 0 and dris_index < NBIa:
            print(f"{df_dris.columns[i]} dris_index > 0 but dris_index < NBIa")
        else:
            print(f"{df_dris.columns[i]} dris_index < 0 but dris_index > NBIa")
            

interpret(DRIS)



NBIa =21.96
I_P dris_index > 0 but dris_index < NBIa
I_Mg dris_index > 0 but dris_index < NBIa
I_N Excess
I_Ca Deficiency-prone
I_Mn Deficiency
